# Extending Guideline

This notebook shows examples of extending the feature of the chmap library, especial for electrode selecting method.
For extending UI components, please see [Extending_View](Extending_View.ipynb).

## Provide another electrode selecting method for the [Neuropixels probe](https://www.neuropixels.org/)

Create a new file, and name it `my_selection.py`.

In [1]:
from chmap.probe_npx import *

def my_selection(desp: NpxProbeDesp, chmap: ChannelMap, s: list[NpxElectrodeDesp], **kwargs) -> ChannelMap:
    """
    Selecting electrodes based on the electrode blueprint.

    :param desp:
    :param chmap: channelmap type. It is a reference.
    :param s: channelmap blueprint
    :param kwargs: other parameters.
    :return: generated channelmap
    """
    pass

Check the function `my_selection` whether can be loaded by the function `load_selector()`.

In [ ]:
from chmap.probe_npx.select import load_select

load_select('module.my_selection:my_selection') # if my_selection.py put under module 'module'

then you can use it in the command-line.

    chmap --selector=module.my_selection:my_selection

Currently providing two implementations:

* `default` (`chmap.probe_npx.select_default:electrode_select`): default selection method.
* `weaker` (`chmap.probe_npx.select_weaker:electrode_select`): same as default, but with a weaker arrangement rule.

You can check the source code for actual runnable code. Furthermore, the notebook [Selecting](Selecting.ipynb) show and explain the simple version of `weaker` selection.

### Implement demostrating

Here use random picking strategy (without considering the electrode's policy) to demostrate how to build a Neuropixels channelmap.

In [4]:
import random

def my_selection(desp: NpxProbeDesp, chmap: ChannelMap, s: list[NpxElectrodeDesp], **kwargs) -> ChannelMap:
    # new an empty channelmap
    ret = desp.new_channelmap(chmap)

    # create a candidate set
    cand = list(s)
    
    while not self.is_valid(ret) and len(cand) > 0: # when the number of selected electrodes below required.
        # random an electrode from the candidate set
        e = random.choice(cand)

        # put e into channelmap
        desp.add_electrode(ret, e)

        # remove invalid electrodes from the candidate set
        inve = desp.invalid_electrodes(ret, e, cand)
        cand = [it for it in cand if it not in inve]
        
    return ret


## Provide supporting for another probe type

Create a new file as same as `probe_mynpx.py`.

In [ ]:
from chmap.probe import ProbeDesp, ElectrodeDesp

class MyElectrodeDesp(ElectrodeDesp):
    ... # extra information
    
class MyProbeDesp(ProbeDesp):
    ... # implement all abstract methods here
    

Due to `ProbeDesp` is an abstract class, you need to implement all abstract methods in `MyProbeDesp`. 
Here use `NpxProbeDesp` to explan each abstract method.

### ElectrodeDesp

It is a simple class that only carry the necessary information for each electrode.

In [10]:
from typing import Any, Hashable, ClassVar
from typing_extensions import Self # for Python < 3.11

class ElectrodeDesp:
    """An electrode interface for GUI interaction between different electrode implementations."""

    x: float  # x position in um
    y: float  # y position in um
    electrode: Hashable  # for identify
    channel: Any  # for display in hover
    state: int = 0
    policy: int = 0

    def copy(self, r: ElectrodeDesp, **kwargs) -> Self: ...
    def __hash__(self): ...
    def __eq__(self, other): ...
    def __str__(self): ...
    def __repr__(self): ...

You don't need to modify it much, actually, unless you create a new UI component that tries to provide more information for each electrode.

In `NpxElectrodeDesp`, we only re-define the actual type for some attributes.

In [ ]:
class NpxElectrodeDesp(ElectrodeDesp):
    electrode: tuple[int, int, int]  # (shank, column, row)
    channel: int

For the 3D probe that electrodes are located in 3D space, attribute `x` and `y` should be the projected coordinated, so it can be shown on the screen, without chaning too much code in GUI part.

### ProbeDesp

#### Class Declaration

The class `ProbeDesp[M, E]` is a generic class that carries two type variables: `M` and `E`, 
where `M` indicates the type of channelmap, and `E` indicates the type of ElectrodeDesp subclass.
For a particular `ProbeDesp[M, E]` implementation, you need to specify these two type variables when declaring.

**Note**: The following code blocks use `NpxProbeDesp` as an example, but all `M` and `E` are kept for demonstrating. 
In actual implementation, they should be replaced with the actual types.

In [16]:
class NpxProbeDesp(ProbeDesp[ChannelMap, NpxElectrodeDesp]):
    ... # skip below

#### Supporting types, electrode states, and selecting policies

The following three properties provide information on what sub-types of supporting probe type, possible electrode state (selected, unselected, or forbidden), and supporting selecting policies. 
The GUI will read the returned dict to generate the corresponding UI controls.

**Predefined states**

* `STATE_UNUSED`: electrode is not used, and it is selectable.
* `STATE_USED`: electrode is selected.
* `STATE_FORBIDDEN`: electrode is not used, but it is not selectable.

**Note** : `STATE_FORBIDDEN` is a valid electrode state, but it is handled by the program instead of users, so it does't need to
present in `possible_states`.

**Predefined policies**

* `POLICY_UNSET`: initial policy value
* `POLICY_SET`: pre-selected
* `POLICY_FORBIDDEN`: never be selected
* `POLICY_REMAINDER`: random selected, less priority

In [17]:
class NpxProbeDesp:
    ... # continue from above
    
    # specific policies for this selecting method.
    POLICY_D1: ClassVar = 11 # full-density selecting policy
    POLICY_D2: ClassVar = 12 # half-density selecting policy
    POLICY_D4: ClassVar = 13 # quarter-density selecting policy
    
    @property
    def supported_type(self) -> dict[str, int]:
        return {'Probe description': probe_code} # where probe_code will be used in new_channelmap(probe_type)
    @property
    def possible_states(self) -> dict[str, int]:
        return {'electrode state description': state_code} # where state_code is either STATE_UNUSED, STATE_USED, or STATE_* etc.
    @property
    def possible_policies(self) -> dict[str, int]:
        return {'electrode policy description': policy_code} # where policy_code is either POLICY_UNSET, POLICY_SET, or POLICY_* etc.

    ... # skip below

#### Custom UI components

You can provide probe-specific UI components. 
`NpxProbeDesp` provides, for example, `ElectrodeDensityDataView` for plotting the electrode density curve along the shanks.

For custom UI components, please check [Provide another Bokeh UI component](#Provide-another-Bokeh-UI-component) section.

In [18]:
from chmap.config import ChannelMapEditorConfig
from chmap.views.base import ViewBase

class NpxProbeDesp:
    ... # continue from above

    def extra_controls(self, config: ChannelMapEditorConfig) -> list[ViewBase | type[ViewBase]]: 
        from chmap.views.data_density import ElectrodeDensityDataView
        return [ElectrodeDensityDataView]
    
    ... # skip below

#### File IO

The following property and methods define what files are look at and how to read/write them from/to disk. 

In [ ]:
class NpxProbeDesp:
    ... # continue from above

    # channelmap file
    @property
    def channelmap_file_suffix(self) -> str:
        return '.imro'
    def load_from_file(self, file: Path) -> M: ...
    def save_to_file(self, chmap: M, file: Path): ...

    # electrode policy
    def electrode_to_numpy(self, s: list[E]) -> NDArray[np.int_]: ...
    def electrode_from_numpy(self, s: list[E], a: NDArray[np.int_]) -> list[E]: ...
    
    ... # skip below

#### Channelmap editing

In [ ]:
class NpxProbeDesp:
    ... # continue from above

    def new_channelmap(self, chmap: int | M) -> M: ...
    def copy_channelmap(self, chmap: M) -> M: ...
    def channelmap_desp(self, chmap: M | None) -> str: ...
    def all_electrodes(self, chmap: int | M) -> list[E]: ...
    def all_channels(self, chmap: M, s: Iterable[E] = None) -> list[E]: ...
    def add_electrode(self, chmap: M, e: E, *, overwrite=False): ...
    def del_electrode(self, chmap: M, e: E): ...

    # not abstract methods

    def get_electrode(self, s: Iterable[E], e: Hashable) -> E | None: ...
    def copy_electrode(self, s: Sequence[E]) -> list[E]: ...
    
    ... # skip below

#### Probe restriction rules

Probe restriction rules are defined in the following two methods. 

**Note**: These two methods should be pure methods that do not contain side effects. 
For example, `probe_rule` doesn't give different results for the same electrodes `e1`, `e2` inputs.
However, if a probe restriction is context-depend, which means the electrode selecting order makes the side effect of `probe_rule`,
there are some ways to do it:

1. record the electrode selecting order in `M`, then `probe_rule` becomes a pure method that its return depends on the `M`. (ignore what `probe_rule`'s document said about `M`)
2. write other methods to support `select_electrodes` correctly.

In [ ]:
class NpxProbeDesp:
    ... # continue from above

    def is_valid(self, chmap: M) -> bool: ...
    def probe_rule(self, chmap: M, e1: E, e2: E) -> bool: ...

    # not abstract methods

    def invalid_electrodes(self, chmap: M, e: E | Iterable[E], s: Iterable[E]) -> list[E]: ...
    
    ... # skip below

#### Electrode selection

**Note**: we keep `kwargs` in the `select_electrodes` signature to provide a way to give extra parameters during electrode selection. 
It can be given from the GUI via `ProbeView.selecting_parameters` attribute (or `ChannelMapEditorApp.probe_view.selecting_parameters`).

In [ ]:
class NpxProbeDesp:
    ... # continue from above

    def select_electrodes(self, chmap: M, s: list[E], **kwargs) -> M: ...
    
    ... # skip below